In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly as py
import plotly.graph_objs as go
import requests
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.utils import *
from datetime import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
import random
import re

Using TensorFlow backend.


In [2]:
file = 'last_of_us.txt'

text = ''
array = []
with open(file, 'r', encoding='UTF-8')  as f:
    lines = f.readlines()
    
    text = text.join([i for i in lines if re.match(r'^[A-Z].*:', i)])

for t in text.split('\n')[:1000]:
    array.append(t)

# 전체에 배열을 뒤져서  겹치지 않는 단어별로 차례대로 숫자를 맥이고 각가의 단어를 숫자로 치환한다..
token = Tokenizer(lower=False, filters='.,?;\'\"-')
token.fit_on_texts(array)
token.word_index  

{'Joel:': 1,
 'Tess:': 2,
 'I': 3,
 's': 4,
 'the': 5,
 'you': 6,
 'Ellie:': 7,
 'to': 8,
 'it': 9,
 't': 10,
 'we': 11,
 're': 12,
 'a': 13,
 'You': 14,
 'this': 15,
 'that': 16,
 'Sarah:': 17,
 'We': 18,
 'of': 19,
 'here': 20,
 'up': 21,
 'out': 22,
 'on': 23,
 'What': 24,
 'get': 25,
 'is': 26,
 'are': 27,
 'me': 28,
 'Tommy:': 29,
 'Let': 30,
 'your': 31,
 'there': 32,
 'm': 33,
 'do': 34,
 'in': 35,
 'go': 36,
 'Oh': 37,
 'they': 38,
 'for': 39,
 'just': 40,
 'Yeah': 41,
 'us': 42,
 'Marlene:': 43,
 'can': 44,
 'It': 45,
 'They': 46,
 'know': 47,
 'with': 48,
 'what': 49,
 'll': 50,
 'Guard:': 51,
 '2:': 52,
 'No': 53,
 'and': 54,
 'Tess': 55,
 'was': 56,
 'gonna': 57,
 'Hey': 58,
 'not': 59,
 'be': 60,
 'How': 61,
 'Joel': 62,
 'way': 63,
 'back': 64,
 'Alright': 65,
 'There': 66,
 'got': 67,
 'see': 68,
 'That': 69,
 'mon': 70,
 'around': 71,
 'don': 72,
 'C': 73,
 'shit': 74,
 'through': 75,
 'This': 76,
 'all': 77,
 'Okay': 78,
 'Look': 79,
 'have': 80,
 'Get': 81,
 'hell': 8

In [3]:
#단어의 갯수
len(token.word_index)

1384

In [4]:
#array있는 String을 seq로 변경한다. (위에 단어장에서)
seq = token.texts_to_sequences(array)

# RNN 입력은 3차원으로 한다.
#(Sample, timestep, feature)
# i am student

In [5]:
# 길이가 부족한것들은 0으로 채워준다.
# 길이가 넘치는것은 뒤에서부터 채우고 버린다.
seq = pad_sequences(seq, maxlen=10)
seq

array([[ 130,  626,    8, ...,  627,  104,    3],
       [   0,    0,    0, ...,    0,   17,   58],
       [   0,    0,    0, ...,    0,    1,  628],
       ...,
       [  57, 1382,   22, ...,   21,   55,  418],
       [   0,    0,    0, ...,    0,    7,  328],
       [   0,    0,    0, ...,    1, 1384,    9]])

In [6]:
X = seq

In [7]:
#X의 맨위에서 한칸을 제거하고 맨밑으로 넣는다. (바로 다음것을 가져오기위해 루프)
Y = np.vstack((X[1:], X[0]))
Y

array([[   0,    0,    0, ...,    0,   17,   58],
       [   0,    0,    0, ...,    0,    1,  628],
       [   0,    0,    0, ...,  111,  266,  331],
       ...,
       [   0,    0,    0, ...,    0,    7,  328],
       [   0,    0,    0, ...,    1, 1384,    9],
       [ 130,  626,    8, ...,  627,  104,    3]])

In [8]:
Y.shape

(1000, 10)

In [9]:
#reshape 한다
#(-1 , timestep, feature)
X = X.reshape(-1, 10, 1)
Y = Y.reshape(-1, 10 ,1)

X

array([[[ 130],
        [ 626],
        [   8],
        ...,
        [ 627],
        [ 104],
        [   3]],

       [[   0],
        [   0],
        [   0],
        ...,
        [   0],
        [  17],
        [  58]],

       [[   0],
        [   0],
        [   0],
        ...,
        [   0],
        [   1],
        [ 628]],

       ...,

       [[  57],
        [1382],
        [  22],
        ...,
        [  21],
        [  55],
        [ 418]],

       [[   0],
        [   0],
        [   0],
        ...,
        [   0],
        [   7],
        [ 328]],

       [[   0],
        [   0],
        [   0],
        ...,
        [   1],
        [1384],
        [   9]]])

In [10]:
X.shape

(1000, 10, 1)

In [11]:
Y = to_categorical(Y)
Y.shape

(1000, 10, 1385)

In [12]:
'''
LSTM에서는
-> -> -> timestep은 기본적으로 오른쪽으로 전달한다.

Bidirectional
<- <- <- <- 양방향으로 전달한다.
'''

model = Sequential()
#input_shape (timestep, feature))
model.add(Bidirectional(SimpleRNN(128, return_sequences=True), input_shape=(10,1)))
model.add(Bidirectional(SimpleRNN(128, return_sequences=True)))
model.add(Dense(1385, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 10, 256)           33280     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 256)           98560     
_________________________________________________________________
dense_1 (Dense)              (None, 10, 1385)          355945    
Total params: 487,785
Trainable params: 487,785
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(X,Y,epochs=100, batch_size=1)


Epoch 1/100
1000/1000 [==============================] - 12s 12ms/step - loss: 5.0091 - accuracy: 0.3235
Epoch 2/100
1000/1000 [==============================] - 10s 10ms/step - loss: 4.6523 - accuracy: 0.3265
Epoch 3/100
1000/1000 [==============================] - 10s 10ms/step - loss: 4.4889 - accuracy: 0.3272
Epoch 4/100
1000/1000 [==============================] - 10s 10ms/step - loss: 4.3414 - accuracy: 0.3257
Epoch 5/100
1000/1000 [==============================] - 10s 10ms/step - loss: 4.1927 - accuracy: 0.3260
Epoch 6/100
1000/1000 [==============================] - 10s 10ms/step - loss: 4.0351 - accuracy: 0.3264
Epoch 7/100
1000/1000 [==============================] - 10s 10ms/step - loss: 3.8884 - accuracy: 0.3255
Epoch 8/100
1000/1000 [==============================] - 11s 11ms/step - loss: 3.7439 - accuracy: 0.3264
Epoch 9/100
1000/1000 [==============================] - 11s 11ms/step - loss: 3.5957 - accuracy: 0.3285
Epoch 10/100
1000/1000 [==============================]

In [15]:
model.save('lasOfUs.model')

# model = load_model('lasOfUs.model')

pred = model.predict(X[:1])
print(pred)

#to_categorycal 함수를 역변하는 함수
pred = np.argmax(pred, axis=2)
pred

[[[8.22660506e-01 3.93153687e-06 1.75020847e-04 ... 4.87086427e-10
   1.92002803e-11 1.09914005e-07]
  [5.68278313e-01 8.13126862e-02 5.65219321e-04 ... 4.95992691e-10
   2.45127334e-12 1.05699826e-07]
  [9.71207201e-01 5.62412431e-04 1.78293885e-05 ... 1.90218910e-10
   1.42978132e-13 1.54789492e-09]
  ...
  [5.49598098e-01 3.31543153e-03 2.91873189e-03 ... 1.41293130e-12
   1.28042089e-07 1.39450860e-08]
  [8.83445045e-06 9.65547923e-04 1.70790622e-06 ... 4.11849388e-10
   5.42682305e-12 1.98567926e-10]
  [8.57730640e-07 3.05052481e-06 2.19608364e-06 ... 9.60790336e-11
   4.43483097e-12 1.83954985e-08]]]


array([[ 0,  0,  0,  0,  0,  0,  0,  0, 17, 58]], dtype=int64)

In [17]:
idx_word = {}
for w in token.word_index:
    idx_word[token.word_index[w]] = w

idx_word

{1: 'Joel:',
 2: 'Tess:',
 3: 'I',
 4: 's',
 5: 'the',
 6: 'you',
 7: 'Ellie:',
 8: 'to',
 9: 'it',
 10: 't',
 11: 'we',
 12: 're',
 13: 'a',
 14: 'You',
 15: 'this',
 16: 'that',
 17: 'Sarah:',
 18: 'We',
 19: 'of',
 20: 'here',
 21: 'up',
 22: 'out',
 23: 'on',
 24: 'What',
 25: 'get',
 26: 'is',
 27: 'are',
 28: 'me',
 29: 'Tommy:',
 30: 'Let',
 31: 'your',
 32: 'there',
 33: 'm',
 34: 'do',
 35: 'in',
 36: 'go',
 37: 'Oh',
 38: 'they',
 39: 'for',
 40: 'just',
 41: 'Yeah',
 42: 'us',
 43: 'Marlene:',
 44: 'can',
 45: 'It',
 46: 'They',
 47: 'know',
 48: 'with',
 49: 'what',
 50: 'll',
 51: 'Guard:',
 52: '2:',
 53: 'No',
 54: 'and',
 55: 'Tess',
 56: 'was',
 57: 'gonna',
 58: 'Hey',
 59: 'not',
 60: 'be',
 61: 'How',
 62: 'Joel',
 63: 'way',
 64: 'back',
 65: 'Alright',
 66: 'There',
 67: 'got',
 68: 'see',
 69: 'That',
 70: 'mon',
 71: 'around',
 72: 'don',
 73: 'C',
 74: 'shit',
 75: 'through',
 76: 'This',
 77: 'all',
 78: 'Okay',
 79: 'Look',
 80: 'have',
 81: 'Get',
 82: 'hell

In [19]:

temp = ''
for i in pred:
    for w in i:
        if w != 0:
            temp += idx_word[w]
            temp += ' '
            
print(temp)

Sarah: Hey 


In [21]:
#X[1] - 이런형태로 지정하면 2차원이기때문에 predict에 바로 넣을수 없다. 그래서 np.expand_dims(X[ran], axis=0) 와 같이  model의 prediction value로 사용할 수 있다.

for i in range(10):
    temp = ''
    ran = random.randrange(0, len(X))
    pred = model.predict(np.expand_dims(X[ran], axis=0))
    
    pred = np.argmax(pred, axis=2)
    
    for i in pred:
        for w in i:
            if w != 0:
                temp += idx_word[w]
                temp += ' '
        temp += '\n'
        print(array[ran])
        print(temp)
    

Tess: Oh shit.
just Get they here! Go! 

Terence: Okay. I can do that. Don't get all huffy-puffy about it.
been awhile You don t visit us any more 

Reporter: We've received reports that victims afflicted with the infection
Sarah: That this nearby 

Tommy: Over there!
fuck Sarah: get people did the fire 

Joel: Here's how this thing's gonna play out. You don't bring up Tess -- ever.
Tess: Ellie: Sure 

Joel: Shit.
Joel: Tess 

Tess: That is a stupid idea.
out Yeah Joel: Well to what 

Sentry 2: Plenty of other smugglers do it. What do you think's gonna happen
Sentry: d still take my chances in here 

Marlene: Good, stay quiet.
Marlene: Okay watch yourselves 

Marlene: Ellie...
my she them 

